In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Column Names are changed manually
table_nm = 'DLAB' 
names = ['iqr_dt','cus_no','act_no','mkt_gb','pdt_gb','iem_cd','iem_krl_anm','bnc_qty','tot_aet_amt'] 

file_path = f'/home/{table_nm}.csv' 
df_trd = pd.read_csv(file_path,  names = names) 
df_trd.head()

In [ ]:
#df_cus 불러오기
table_nm = 'DLAB2' 
names = ['cus_no','sex_dit_cd','cus_age','zip_ctp_cd', 'tco_cus_grd_cd', 'ivs_icn_cd'] 

file_path = f'/home/{table_nm}.csv' 
df_cus = pd.read_csv(file_path,  names = names) 
df_cus.head()

In [ ]:
#df_act 불러오기
table_nm = 'DLAB3' 
names = ['act_no','cus_no','act_opn_ym'] 

file_path = f'/home/{table_nm}.csv' 
df_act = pd.read_csv(file_path,  names = names) 
df_act.head()

In [6]:
# Making and cleaning data set to cluster
df_bal = df_trd.copy()[['iqr_dt', 'cus_no', 'tot_aet_amt']].groupby(['iqr_dt', 'cus_no']).sum()
df_bal = df_bal.reset_index()

df_bal_kor = df_trd[df_trd['mkt_gb'] == '국내'].copy()[['iqr_dt', 'cus_no', 'tot_aet_amt']].groupby(['iqr_dt', 'cus_no']).sum()
df_bal_kor = df_bal_kor.reset_index()

df_bal_ovs = df_trd[df_trd['mkt_gb'] == '해외'].copy()[['iqr_dt', 'cus_no', 'tot_aet_amt']].groupby(['iqr_dt', 'cus_no']).sum()
df_bal_ovs = df_bal_ovs.reset_index()

df_bal_tot = pd.merge(left=df_bal, 
                      right=df_bal_kor[['iqr_dt','cus_no', 'tot_aet_amt']], 
                      on=['cus_no', 'iqr_dt'], 
                      suffixes=('', '_kr'), 
                      how='left')
df_bal_tot = pd.merge(left=df_bal_tot, 
                      right=df_bal_ovs[['iqr_dt','cus_no', 'tot_aet_amt']], 
                      on=['cus_no', 'iqr_dt'], 
                      suffixes=('', '_ov'), 
                      how='left')

df_bal_tot[['tot_aet_amt_kr', 'tot_aet_amt_ov']] = df_bal_tot[['tot_aet_amt_kr', 'tot_aet_amt_ov']].fillna(0)

bank_cus = pd.merge(left=df_cus, right = df_bal_tot, on='cus_no', suffixes=('',''))
bank_cus['ovs_ratio'] = bank_cus['tot_aet_amt_ov']/bank_cus['tot_aet_amt']

In [7]:
df_bal_list = []
df_bal_len = []
df_cus_w = df_cus.copy()[(df_cus['cus_no'].isin(df_trd['cus_no'])) & ~(df_cus['cus_no'].isin(df_trd[(df_trd['iem_cd'] == "US25459W4583")]['cus_no']))]
for i in df_bal_tot['iqr_dt'].unique():
    df = df_bal_tot[df_bal_tot['iqr_dt'] == i]
    df = pd.merge(left=df_cus_w, right=df, on='cus_no', suffixes=('','')) #if how='left', customers who haven't opened an account appear
    df['ovs_ratio'] = df['tot_aet_amt_ov'] / df['tot_aet_amt']
    df_bal_list.append(df)
    df_bal_len.append(len(df))

In [8]:
df_bal_list_10_19 = []
df_bal_list_20_24 = []
df_bal_list_25_30 = []
for i in range(len(df_bal_list)):
    df_bal_list_10_19.append(df_bal_list[i][(df_bal_list[i]['cus_age'] > 9) & (df_bal_list[i]['cus_age'] < 20)])
    df_bal_list_20_24.append(df_bal_list[i][(df_bal_list[i]['cus_age'] > 19) & (df_bal_list[i]['cus_age'] < 25)])
    df_bal_list_25_30.append(df_bal_list[i][(df_bal_list[i]['cus_age'] > 24) & (df_bal_list[i]['cus_age'] < 31)])

In [9]:
#20대 초반 투자자 해외투자 비중 변화
#20대 초반 투자자 해외투자액 변화

df_list = df_bal_list_20_24
num_ppl = []
ovs_pct = []
ovs_pct_ratio = []
ovs_avg_ratio = []
ovs_tot_amt = []
for i in range(182):
    df = df_list[i]
    df_hlf = df[df['ovs_ratio'] > 0.5]
    df_ovs = df[df['ovs_ratio'] > 0]
    ovs_pct.append(len(df[df['ovs_ratio'] > 0])/len(df))
    ovs_pct_ratio.append(len(df_hlf)/len(df_ovs))
    ovs_avg_ratio.append(df_ovs['ovs_ratio'].mean())
    ovs_tot_amt.append(df['tot_aet_amt_ov'].sum())

In [ ]:
# Combining the dataset into one dataframe
df_bank_cus = pd.concat([df_bal_list_10_19[181],df_bal_list_20_24[181],df_bal_list_25_30[181]], axis=0)
df_bank_cus

In [101]:
df_trd[['bnc_qty','cus_no','iqr_dt','iem_krl_anm']].groupby(['cus_no','iqr_dt','iem_krl_anm']).count()
print()

In [ ]:
# df_trd 테이블 조회 및 그룹
df_trd_copy = df_trd.copy()
df_test = df_trd_copy[df_trd_copy['bnc_qty'] !=  ' Inc.']
bnc_int = df_test['bnc_qty'].astype(int)

df_test['bnc_qty'] = bnc_int
counting = df_test[['bnc_qty','cus_no','iqr_dt','iem_krl_anm']].groupby(['cus_no','iqr_dt','iem_krl_anm']).sum()

df_trd.head()

In [ ]:
# cus_no 별 bnc_qty 조회
# Column Names are changed manually
countings = df_test[['bnc_qty','cus_no','iqr_dt']].groupby(['cus_no','iqr_dt']).sum()

getting_index = df_test[['cus_no','iqr_dt']].groupby(['cus_no']).count()
index_countings = getting_index.index

In [14]:
# 매매빈도를 위한 helper function 
# bnc_qty의 값이 다르면 counter 세기
def counter(a,b):
    count = 0
    for i in range(a,b,1):
        if i < (b-a)-1:
            if countings['bnc_qty'][i] != countings['bnc_qty'][i+1]:
                count += 1
    return count

lst = [0]
for i in range(getting_index.shape[0]):
    lst.append(getting_index['iqr_dt'].iloc[i])

In [18]:
# Helper function 인 counter를 이용한 매매빈도 list 추출
res = []
for i in range(len(lst)):
    if i < len(lst)-1:
        n = lst[i] + lst[i+1]
        val = counter(lst[i], n)
    res.append(val)
    
res

In [100]:
# 주식 매매빈도를 구하기 위한 정리
df_trd.head()
testing = df_trd.copy()
vals = []
for i in range(testing.shape[0]):
    vals.append(1)

testing['ones'] = vals

countings = df_test[['bnc_qty','cus_no','iqr_dt']].groupby(['cus_no','iqr_dt']).sum()

getting_index = df_test[['cus_no','iqr_dt']].groupby(['cus_no']).count()
index_countings = getting_index.index

testings = testing[['ones','cus_no','iqr_dt']].groupby(['cus_no','iqr_dt']).count()

In [ ]:
# 주식 매매빈도 그래프
getting_index['count'] = res[:440]
getting_index

counting_trade_num = getting_index.drop(columns = 'iqr_dt')
counting_trade_num.hist(bins=15)

In [ ]:
# Making and cleaning data set to cluster
df_bal = df_trd.copy()[['iqr_dt', 'cus_no', 'tot_aet_amt']].groupby(['iqr_dt', 'cus_no']).sum()
df_bal = df_bal.reset_index()

df_bal_kor_stk = df_trd[(df_trd['mkt_gb'] == '국내') & (df_trd['pdt_gb'] == '주식')].copy()[['iqr_dt', 'cus_no', 'tot_aet_amt']].groupby(['iqr_dt', 'cus_no']).sum()
df_bal_kor_stk = df_bal_kor_stk.reset_index()
df_bal_ovs_stk = df_trd[(df_trd['mkt_gb'] == '해외') & (df_trd['pdt_gb'] == '주식')].copy()[['iqr_dt', 'cus_no', 'tot_aet_amt']].groupby(['iqr_dt', 'cus_no']).sum()
df_bal_ovs_stk = df_bal_ovs_stk.reset_index()

df_bal_kor_etf = df_trd[(df_trd['mkt_gb'] == '국내') & (df_trd['pdt_gb'] == 'ETF')].copy()[['iqr_dt', 'cus_no', 'tot_aet_amt']].groupby(['iqr_dt', 'cus_no']).sum()
df_bal_kor_etf = df_bal_kor_etf.reset_index()
df_bal_ovs_etf = df_trd[(df_trd['mkt_gb'] == '해외') & (df_trd['pdt_gb'] == 'ETF')].copy()[['iqr_dt', 'cus_no', 'tot_aet_amt']].groupby(['iqr_dt', 'cus_no']).sum()
df_bal_ovs_etf = df_bal_ovs_etf.reset_index()


df_bal_tot = pd.merge(left=df_bal, 
                      right=df_bal_kor_stk[['iqr_dt','cus_no', 'tot_aet_amt']], 
                      on=['cus_no', 'iqr_dt'], 
                      suffixes=('', '_kor_stk'), 
                      how='left')
df_bal_tot = pd.merge(left=df_bal_tot, 
                      right=df_bal_ovs_stk[['iqr_dt','cus_no', 'tot_aet_amt']], 
                      on=['cus_no', 'iqr_dt'], 
                      suffixes=('', '_ovs_stk'), 
                      how='left')
df_bal_tot = pd.merge(left=df_bal_tot, 
                      right=df_bal_kor_etf[['iqr_dt','cus_no', 'tot_aet_amt']], 
                      on=['cus_no', 'iqr_dt'], 
                      suffixes=('', '_kor_etf'), 
                      how='left')
df_bal_tot = pd.merge(left=df_bal_tot, 
                      right=df_bal_ovs_etf[['iqr_dt','cus_no', 'tot_aet_amt']], 
                      on=['cus_no', 'iqr_dt'], 
                      suffixes=('', '_ovs_etf'), 
                      how='left')

df_bal_tot[['tot_aet_amt_kor_stk', 'tot_aet_amt_ovs_stk', 'tot_aet_amt_kor_etf', 'tot_aet_amt_ovs_etf']] = df_bal_tot[['tot_aet_amt_kor_stk', 'tot_aet_amt_ovs_stk', 'tot_aet_amt_kor_etf', 'tot_aet_amt_ovs_etf']].fillna(0)

df_bal_list = []
df_bal_len = []
for i in df_bal_tot['iqr_dt'].unique():
    df = df_bal_tot[df_bal_tot['iqr_dt'] == i]
    df = pd.merge(left=df_cus_w, right=df, on='cus_no', suffixes=('','')) #if how='left', customers who haven't opened an account appear
    df['kor_etf_ratio'] = df['tot_aet_amt_kor_etf'] / (df['tot_aet_amt_kor_stk']+df['tot_aet_amt_kor_etf'])
    df['ovs_etf_ratio'] = df['tot_aet_amt_ovs_etf'] / (df['tot_aet_amt_ovs_stk']+df['tot_aet_amt_ovs_etf'])
    df[['kor_etf_ratio', 'ovs_etf_ratio']] = df[['kor_etf_ratio', 'ovs_etf_ratio']].fillna(0)
    df_bal_list.append(df)
    df_bal_len.append(len(df))
    
df_getting = df_bal_list[181]
df_getting['cus_no']

In [ ]:
# Adding Count column to the modified data
counting_trade = counting_trade_num.reset_index()
count = counting_trade[counting_trade['cus_no'].isin(df_getting['cus_no'])]
count = count.set_index('cus_no')
count_lst = []
for i in count['count']:
    count_lst.append(i)
len(count_lst)


In [ ]:
# Final df_bank_cus data set to cluster 
ten_thirty = df_bal_list[181].set_index(keys='cus_no')
df_bank_cus_copy = ten_thirty.drop(['ivs_icn_cd', 'iqr_dt', 'zip_ctp_cd'], axis=1)
df_bank_cus_copy = df_bank_cus_copy.reset_index().sort_values(by='cus_no')
df_bank_cus_copy['count'] = count_lst
df_bank_cus_copy = df_bank_cus_copy.set_index('cus_no')
df_bank_cus_copy.head()

In [51]:
from kmodes.kmodes import KModes

In [ ]:
# Choosing K 
cost = []
for clusters in list(range(1,10)):
    modes = KModes(n_clusters=clusters, init = "Random", n_init=5, verbose=1)
    modes.fit_predict(df_bank_cus_copy)
    cost.append(modes.cost_)

In [ ]:
graph = np.array([i for i in range (1,10,1)])
plt.plot(graph, cost)

In [ ]:
# KModes clustering ran 10 times with K=8
km = KModes(n_clusters=8, init="Random", n_init=10, verbose=1)
fit_clusters = km.fit_predict(df_bank_cus_copy)


In [ ]:
# Input the predicted group into original data
df_bank_cus_copy['cluster'] = fit_clusters
df_bank_cus_copy.head()

In [ ]:
#cluster_0 top 5
cluster_0 = df_bank_cus_copy[df_bank_cus_copy['cluster'] == 0].reset_index()
table_0 = df_trd[(df_trd['iqr_dt'] == 20210701) & (df_trd['cus_no'].isin(cluster_0['cus_no']))].groupby('iem_krl_anm').count().sort_values(by='iqr_dt', ascending=False).head()
table_0 = table_0.drop(columns =['iqr_dt', 'cus_no', 'act_no','mkt_gb','pdt_gb','iem_cd','tot_aet_amt'])
table_0

In [ ]:
#cluster_1 top 5
cluster_1 = df_bank_cus_copy[df_bank_cus_copy['cluster'] == 1].reset_index()
table_1 = df_trd[(df_trd['iqr_dt'] == 20210701) & (df_trd['cus_no'].isin(cluster_1['cus_no']))].groupby('iem_krl_anm').count().sort_values(by='iqr_dt', ascending=False).head()
table_1 = table_1.drop(columns =['iqr_dt', 'cus_no', 'act_no','mkt_gb','pdt_gb','iem_cd','tot_aet_amt'])
table_1

In [ ]:
#cluster_2 top 5
cluster_2 = df_bank_cus_copy[df_bank_cus_copy['cluster'] ==2].reset_index()
df_trd[(df_trd['iqr_dt'] == 20210701) & (df_trd['cus_no'].isin(cluster_2['cus_no']))].groupby('iem_krl_anm').count().sort_values(by='iqr_dt', ascending=False).head()

In [ ]:
#cluster_3 top 5
cluster_3 = df_bank_cus_copy[df_bank_cus_copy['cluster'] == 3].reset_index()
df_trd[(df_trd['iqr_dt'] == 20210701) & (df_trd['cus_no'].isin(cluster_3['cus_no']))].groupby('iem_krl_anm').count().sort_values(by='iqr_dt', ascending=False).head()

In [ ]:
# cluster_4 top 5 
cluster_4 = df_bank_cus_copy[df_bank_cus_copy['cluster'] == 4].reset_index()
df_trd[(df_trd['iqr_dt'] == 20210701) & (df_trd['cus_no'].isin(cluster_4['cus_no']))].groupby('iem_krl_anm').count().sort_values(by='iqr_dt', ascending=False).head()

In [67]:
# Naming cluster 5,6,7
cluster_5 = df_bank_cus_copy[df_bank_cus_copy['cluster'] == 5].reset_index()
cluster_6 = df_bank_cus_copy[df_bank_cus_copy['cluster'] == 6].reset_index()
cluster_7 = df_bank_cus_copy[df_bank_cus_copy['cluster'] == 7].reset_index()

In [ ]:
#Getting the mean val of all the columns to check each group's features
df_used = df_bank_cus_copy.groupby('cluster').mean()
df_ovs = (df_used['tot_aet_amt_ovs_etf'] + df_used['tot_aet_amt_ovs_stk'])/df_used['tot_aet_amt']
df_used['ovs_ratio'] = df_ovs
temp = df_used['count']
df_used = df_used.drop(columns=['count'])
df_used['count'] = temp
df_used